In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


In [8]:
import getpass
import os

os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import AzureChatOpenAI
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://gpt4edu-ncus.openai.azure.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "GPT4o"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-12-01-preview"
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [5]:
import getpass
import os

os.environ["Ir_API_KEY"] = getpass.getpass()

# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     openai_api_base="https://api.wlai.vip/v1",
#     openai_api_key=os.environ["Ir_API_KEY"],
# )

In [10]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
os.environ["TOKENIZERS_PARALLELISM"]="false"
# Load, chunk and index the contents of the blog.
pdf = '/home/v-haoguan/yuxuan/scihub.py/scihub/EMPIAR-10164.pdf'
loader = PyPDFLoader(
file_path=pdf,
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
from langchain_openai import OpenAIEmbeddings
# embedding = OpenAIEmbeddings(base_url="https://api.wlai.vip/v1", api_key=os.environ["Ir_API_KEY"])
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings(model_name="infgrad/stella_en_1.5B_v5"))
# vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
from langchain.prompts import PromptTemplate

# Modify the prompt template
custom_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. Focus on the sections labeled 'Methods' and 'Results' for key details. Provide a detailed description focusing on system setup, resources used, specific characteristics observed, preparations made, and any additional relevant findings.

Context:
{context}

Question:
{question}

Answer:
"""
)

# Use the custom prompt in your RAG chain
# prompt = custom_prompt_template
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Invoke with the specific question
# docs 
# retriever.invoke("summarize the article for the obtained tomography's caption. It must contain the system, the resources , the characteristics, and the preparations. And more information can throughly describe the tomography. It's the direct information more important, not the summarized and induced supposition and conclusion")
rag_chain.invoke("summarize the article for the obtained tomography's caption. It must contain the system, the resources , the characteristics, and the preparations. And more information can throughly describe the tomography. It's the direct information more important, not the summarized and induced supposition and conclusion")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'The article describes a study using cryo-electron microscopy (cryo-EM) to resolve structures essential for immature HIV-1 virus assembly. The system involved tomography acquisition schemes implemented by WHJH, with technical support from the EMBL IT services unit and the EMBL proteomics core facility. Samples were prepared by FKMS and MO, and representative tomograms and final EM structures have been deposited in the EMDB, while the refined HIV-1 CA-SP1 model is in the PDB.'

In [19]:
import os
import getpass
import requests
from langchain_community.document_loaders import PyPDFLoader

# Prompt for API key securely

# Load environment variables
azure_endpoint = "https://gpt4edu-ncus.openai.azure.com/"
azure_deployment = "GPT4o"
api_version = "2023-12-01-preview"
api_key = os.environ["AZURE_OPENAI_API_KEY"]

# Construct the full URL
url = f"{azure_endpoint}openai/deployments/{azure_deployment}/chat/completions?api-version={api_version}"

# Headers
headers = {
    "Content-Type": "application/json",
    "api-key": f"{api_key}"
}
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
pdf = '/home/v-haoguan/yuxuan/scihub.py/scihub/EMPIAR-12082.pdf'
loader = PyPDFLoader(
file_path=pdf,
)
docs = loader.load()
formatted_docs = format_docs(docs)
question = f"""
Please extract key information describing Cryo-ET image data from the following article. The information should include a system description, scale, focus point description, imaging environment and preparation method, data acquisition conditions, resolution and minimum pixel size, labeling and staining (if applicable), and image processing and reconstruction. Provide the extracted information in the following format:

1. System Description:
   - The type of structure or sample shown in the image, such as protein, cell type, and specific site, as well as the chemical or biological environment of the observed system.

2. Scale:
   - The size of the system and the scale of the information being observed.

3. Focus Point Description:
   - If it’s a protein, what protein and its site; if it’s a cell, describe the type and specific site.

4. Imaging Environment and Preparation Method:
   - The environment in which the sample was imaged, such as temperature, natural state, and sample handling.

5. Data Acquisition Conditions:
   - Type of microscope, accelerating voltage, sample tilt angle range, and other data acquisition parameters.

6. Resolution and Minimum Pixel Size:
   - The resolution and minimum pixel size of the image.

7. Labeling and Staining (if applicable):
   - Whether any labeling or staining techniques were used and their types.

8. Image Processing and Reconstruction:
   - Methods of image processing and 3D reconstruction, such as algorithms or software tools used.

Below is the article content:
{formatted_docs}

Please extract the information and provide it in the specified format.
"""
# Request payload
payload = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"""
{question}

Answer:"""
         }
    ],
    # "max_tokens": 
}

# Make the request
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    response_data = response.json()
    print("Response:", response_data['choices'][0]['message']['content'])


Response: 1. **System Description**:
   - The sample comprises β-amyloid and tau proteins from a postmortem Alzheimer's disease (AD) brain. Specifically, it involves extracellular β-amyloid plaques and intracellular and extracellular tau deposits within the mid-temporal gyrus of the brain.

2. **Scale**:
   - The imaging details includes features on the scale of nanometers. β-amyloid fibrils and tau filaments are measured to be between 3–12 nm in diameter.

3. **Focus Point Description**:
   - Focus is on β-amyloid plaques and tau inclusions. β-amyloid plaques are composed of fibrils, branched structures, protofilament-like rods, extracellular vesicles, cuboidal particles, and other macromolecular constituents. Tau inclusions are composed of unbranched tau filaments forming clusters.

4. **Imaging Environment and Preparation Method**:
   - Samples were obtained from rapid autopsy postmortem brain tissue, including a 6-hour postmortem interval. After thawing, acute brain slices were lab

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass(prompt="Enter your Azure OpenAI API Key: ")


In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
pdf = '/home/v-haoguan/yuxuan/scihub.py/scihub/paper_HIV.pdf'
loader = PyPDFLoader(
file_path=pdf,
)
docs = loader.load()
formatted_docs = format_docs(docs)
formatted_docs

" \n  \n  REPORTS  \n \nCite as: Schur  et al., Science  \n10.1126/science.aaf9620 (2016).  \n The major structural protein of HIV -1, Gag, oligomerizes at \nthe plasma membrane of infe cted cells, leading to me m-\nbrane bending and release of immature virus particles. O r-\ndered cleavage of the Gag polyprotein at five sites by the viral protease (PR) then causes a dramatic structural rea r-\nrangement of the virus to the mature, infectious virion (fig. S1, A to C). Gag –Gag interactions in the immature virus are \nmediated by the CA domains and the adjacent SP1 region. After maturation, CA forms a conical core encapsulating the viral RNA -nucleoprotein complex ( 1). The final proteolytic \ncleavage in Gag  occurs between CA and SP1. Even small \nremnants of uncleaved CA -SP1 have a dominant -negative \neffect on infectivity ( 2, 3). Cleavage at this site is inhibited \nby MIs ( 4) (fig. S1B), and polymorphisms in this region \ncaused  resistance against the first -in-class MI Beviri

In [5]:
import os
import getpass
import requests

# Prompt for API key securely
api_key = getpass.getpass(prompt="Enter your OpenAI API Key: ")

# OpenAI API endpoint
url = "https://api.wlai.vip/v1/chat/completions"

# Headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Request payload
payload = {
    "model": "gpt-3.5-turbo",  # or "gpt-3.5-turbo" if you're using GPT-3.5
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, can you help me set up a film club?"}
    ],
    "max_tokens": 100
}

# Make the request
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    response_data = response.json()
    print("Response:", response_data['choices'][0]['message']['content'])
else:
    print("Failed to get a response:", response.status_code, response.text)


Response: Of course! Setting up a film club can be a lot of fun. Here are some steps and considerations to get you started:

### 1. **Define Your Vision and Goals:**
   - **Purpose:** Decide why you want to start the film club. Is it for entertainment, education, or a mix of both?
   - **Audience:** Determine who your target audience is. Is it for students, professionals, a specific age group, etc.?

### 2. **Gather Interested Individuals:**
   - Spread the word among friends, colleagues, or through social media to find people who share your interest in films.
   - Consider creating a simple online form to gauge interest and gather contact information.

### 3. **Choose a Format:**
   - Decide how often you will meet (e.g., weekly, bi-weekly).
   - Determine the duration of each meeting (e.g., 1-2 hours).

### 4. **Select Films:**
   - Establish a method for selecting films (voting, rotating picks, themed months, etc.).
   - Consider diversity in genres, languages, and eras to cater to 